# Imports and paths

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
from pathlib import Path

In [ ]:
npy_file = ("/projects/synsight/data/website_data/jump_compounds_matrix.npy")
METADATA_FILE = ("/projects/synsight/data/website_data/jump_compounds_matrix_metadata.parquet")
output_file = '/projects/synsight/data/website_data/nearest_neighbors.h5'  # Output HDF5 file

In [ ]:
models = ["openphenom", "chada", "resnet50", "dinov2_g"]

In [ ]:
all_embeddings = {}

In [ ]:
for model in models:
    npy_file = Path(f'/projects/synsight/data/jump_embeddings/compounds_embeddings/{model}/Embeddings_norm.npy')
    all_embeddings[model] = np.load(npy_file)
    print(all_embeddings[model].shape)

# Histograms

In [ ]:


# Charger la matrice depuis le fichier .npy
matrix = np.load(DATA_FILE)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing as mp
from pathlib import Path
from tqdm import tqdm

def worker_sample_chunk(args):
    """Compute the true cosine similarity for a subset of rows and flatten the results."""
    row_indices, embeddings = args
    
    # If embeddings are not guaranteed normalized, we compute norms on the fly
    # This can be expensive. Another approach is to ensure embeddings are normalized once at load time.
    
    # For each row in row_indices, compute dot product with all embeddings
    # and divide by norms => true cosine similarity
    row_vecs = embeddings[row_indices]  # shape (len(row_indices), D)
    row_norms = np.linalg.norm(row_vecs, axis=1, keepdims=True)  # shape (len(row_indices), 1)
    all_norms = np.linalg.norm(embeddings, axis=1)  # shape (N,)
    
    # (len(row_indices), D) dot (D, N) => (len(row_indices), N)
    dot_products = np.dot(row_vecs, embeddings.T)
    # divide each row by row_norm * all_norms
    # row_norms is broadcasted across columns, all_norms is broadcasted across rows
    # shape => (len(row_indices), N)
    denom = row_norms * all_norms[np.newaxis, :]
    cosine_sim = dot_products / denom
    
    return cosine_sim.flatten()

def compute_pairwise_sample(embeddings, sample_indices, n_processes):
    """
    Splits the sampled row indices among n_processes,
    computes the cosine similarity in parallel, and returns all pairwise values.
    """
    chunks = np.array_split(sample_indices, n_processes)
    args_list = [(chunk, embeddings) for chunk in chunks if len(chunk) > 0]
    with mp.Pool(n_processes) as pool:
        results = pool.map(worker_sample_chunk, args_list)
    # Concatenate all flattened arrays
    all_values = np.concatenate(results)
    return all_values

if __name__ == '__main__':
    # Dictionary for embeddings
    all_embeddings = {}
    models = ["openphenom", "chada", "resnet50", "dinov2_g"]

    # Load each .npy file as float16 to reduce memory usage
    for model in models:
        npy_file = Path(f'/projects/synsight/data/jump_embeddings/compounds_embeddings/{model}/Embeddings_norm.npy')
        all_embeddings[model] = np.load(npy_file).astype(np.float32)
        # If the file is truly normalized, you could keep it as float16,
        # but float32 is safer for the division.

    # Number of CPU cores
    n_processes = mp.cpu_count()

    # Fraction of rows to sample
    sample_fraction = 0.001  # 0.1% (adjust as needed for memory/speed tradeoff)

    for key, embeddings in tqdm(all_embeddings.items(), desc="Models"):
        N = embeddings.shape[0]
        sample_size = max(1, int(sample_fraction * N))
        sample_indices = np.random.choice(N, size=sample_size, replace=False)

        # Compute pairwise *cosine similarities* for these sampled rows
        all_values = compute_pairwise_sample(embeddings, sample_indices, n_processes)

        # Plot with fewer bins + KDE
        plt.figure(figsize=(8, 6), dpi=300)
        sns.histplot(all_values, bins=200, kde=True, color='blue', alpha=0.3)
        plt.title(f'Distribution of Cosine Similarities for {key}')
        plt.xlabel('Cosine Similarity')
        plt.ylabel('Frequency')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.savefig(f'histogram_kde_{key}_linear.png', dpi=300)
        plt.close()

        # Log-scale version
        plt.figure(figsize=(8, 6), dpi=300)
        sns.histplot(all_values, bins=200, kde=True, color='blue', alpha=0.3, log_scale=(False, True))
        plt.title(f'Distribution of Cosine Similarities (log scale) for {key}')
        plt.xlabel('Cosine Similarity')
        plt.ylabel('Frequency (log scale)')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.savefig(f'histogram_kde_{key}_log.png', dpi=300)
        plt.close()


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing as mp
from pathlib import Path
from tqdm import tqdm

def worker_sample_chunk(args):
    """Compute the true cosine similarity for a subset of rows and flatten the results."""
    row_indices, embeddings = args
    row_vecs = embeddings[row_indices]  # shape (len(row_indices), D)
    row_norms = np.linalg.norm(row_vecs, axis=1, keepdims=True)  # shape (len(row_indices), 1)
    all_norms = np.linalg.norm(embeddings, axis=1)  # shape (N,)
    dot_products = np.dot(row_vecs, embeddings.T)  # shape (len(row_indices), N)
    denom = row_norms * all_norms[np.newaxis, :]
    cosine_sim = dot_products / denom
    return cosine_sim.flatten()

def compute_pairwise_sample(embeddings, sample_indices, n_processes):
    """
    Splits the sampled row indices among n_processes,
    computes the cosine similarity in parallel, and returns all pairwise values.
    """
    chunks = np.array_split(sample_indices, n_processes)
    args_list = [(chunk, embeddings) for chunk in chunks if len(chunk) > 0]
    with mp.Pool(n_processes) as pool:
        results = pool.map(worker_sample_chunk, args_list)
    all_values = np.concatenate(results)
    return all_values

if __name__ == '__main__':
    # Dictionary for embeddings
    all_embeddings = {}
    models = ["openphenom", "chada", "resnet50", "dinov2_g"]

    # Load each .npy file as float32 to ensure safe division (even if originally float16)
    for model in models:
        npy_file = Path(f'/projects/synsight/data/jump_embeddings/compounds_embeddings/{model}/Embeddings_norm.npy')
        all_embeddings[model] = np.load(npy_file).astype(np.float32)

    # Number of CPU cores for parallel processing
    n_processes = mp.cpu_count()

    # Fraction of rows to sample (adjust for memory/speed trade-off)
    sample_fraction = 0.001  # 0.1%

    # Dictionary to store distributions for each model
    distributions = {}

    # Compute cosine similarity distributions for each model
    for key, embeddings in tqdm(all_embeddings.items(), desc="Models"):
        N = embeddings.shape[0]
        sample_size = max(1, int(sample_fraction * N))
        sample_indices = np.random.choice(N, size=sample_size, replace=False)
        all_values = compute_pairwise_sample(embeddings, sample_indices, n_processes)
        distributions[key] = all_values

    ## Create a combined plot for all models (linear scale)
    #plt.figure(figsize=(10, 8), dpi=300)
    #for model, values in distributions.items():
    #    # Plot histogram and KDE for each model; using stat="density" normalizes the histograms
    #    sns.histplot(values, bins=200, stat="density", alpha=0.3, kde=True, label=model)
    #
    #plt.title("Distribution of Cosine Similarities for All Models")
    #plt.xlabel("Cosine Similarity")
    #plt.ylabel("Density")
    #plt.legend(loc='upper right')
    #plt.grid(axis='y', linestyle='--', alpha=0.7)
    #plt.tight_layout()
    #plt.savefig("histogram_kde_all_models_linear.png", dpi=300)
    #plt.close()

    # (Optional) Create a log-scale version for the y-axis
    # Create a log-scale plot with filled KDE curves for each model
    plt.figure(figsize=(10, 8), dpi=300)
    for model, values in distributions.items():
        # Plot a filled KDE for each model
        sns.kdeplot(values, fill=True, alpha=0.3, label=model)
        
    # Set y-axis to log scale (note: this applies to the density values computed by the KDE)
    plt.yscale('log')
    plt.title("Distribution of Cosine Similarities (log scale) with Filled KDE")
    plt.xlabel("Cosine Similarity")
    plt.ylabel("Density (log scale)")
    plt.legend(loc='upper right')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig("filled_kde_all_models_log.png", dpi=300)
    plt.close()



In [ ]:
#!/usr/bin/env python
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm
import math

def compute_block(start, end, normalized_embeddings):
    """
    Computes a block of the cosine distance matrix for rows [start, end).
    Cosine distance is defined as: distance = 1 - cosine_similarity.
    """
    # Extract the block (shape: block_size x n_features)
    block = normalized_embeddings[start:end]
    # Compute cosine similarity between the block and all embeddings
    # (resulting shape: (block_size, n_samples))
    similarity = np.dot(block, normalized_embeddings.T)
    # Convert cosine similarity to cosine distance
    distances = 1.0 - similarity
    # Cast to smallest precision (float16)
    return distances.astype(np.float16)

def main():
    # Load embeddings: shape (n_samples, n_features)
    embeddings = all_embeddings["dinov2_g"]
    n_samples, _ = embeddings.shape

    # Normalize the embeddings (L2 normalization)
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    normalized_embeddings = embeddings / norms

    # Choose a block size (adjust as needed to manage memory)
    block_size = 100  # For example, 500 rows per block
    n_blocks = math.ceil(n_samples / block_size)
    
    print(f"Computing distance matrix for {n_samples} samples in {n_blocks} blocks...")
    
    # Compute the distance matrix in parallel over blocks
    results = Parallel(n_jobs=-1, backend="threading")(
        delayed(compute_block)(
            start, min(start + block_size, n_samples), normalized_embeddings
        )
        for start in tqdm(range(0, n_samples, block_size), desc="Processing blocks")
    )
    
    # Stack all computed blocks vertically to form the full matrix
    distance_matrix = np.vstack(results)
    
    # Save the distance matrix as a .npy file with float16 precision
    np.save("distance_matrix.npy", distance_matrix)
    print("Distance matrix saved to 'distance_matrix.npy'")

if __name__ == "__main__":
    main()


In [ ]:

# Afficher la distribution des valeurs avec un histogramme
plt.figure(figsize=(8, 6))
plt.hist(all_values, bins=5000, color='blue', alpha=0.3)
plt.title('Distribution des valeurs dans la matrice')
plt.xlabel('Valeurs')
plt.ylabel('Fréquence')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Precompute distances

In [ ]:
m = 1000  # Number of nearest neighbors

In [ ]:
# Load metadata
metadata = pd.read_parquet(METADATA_FILE)
metadata_ids = metadata['Metadata_JCP2022'].values  # Unique molecule IDs


In [ ]:
metadata_ids

In [ ]:
matrix = np.load(npy_file)

In [ ]:
def process_row(i):
    """
    Process a single row of the distance matrix to find the m closest neighbors
    and include the distance to a specific molecule.

    Args:
        i (int): Index of the row in the distance matrix.

    Returns:
        tuple: (molecule_id, closest_ids, closest_distances)
    """
    distances = matrix[i]

    # Distance to the target molecule (JCP2022_033924)
    target_index = np.where(metadata_ids == 'JCP2022_033924')[0][0]
    dmso_distance = distances[target_index]

    # Find m closest molecules using partial sorting (excluding self if needed)
    closest_indices = np.argpartition(distances, m)[1:m+1]  # Top m indices (unsorted)
    closest_distances = distances[closest_indices]

    # Sort these m indices to ensure proper order
    sorted_indices_within_chunk = np.argsort(closest_distances)
    closest_indices = closest_indices[sorted_indices_within_chunk]
    closest_distances = closest_distances[sorted_indices_within_chunk]

    # Get IDs for the closest molecules
    closest_ids = metadata_ids[closest_indices]

    # Return the results
    return metadata_ids[i], closest_ids, closest_distances, dmso_distance

# Parallel processing
with Pool(processes=cpu_count()) as pool:
    # Use tqdm for progress tracking
    results = list(tqdm(pool.imap(process_row, range(matrix.shape[0])), total=matrix.shape[0]))






In [ ]:
# Save results to HDF5
with h5py.File(output_file, 'w') as h5f:
    for molecule_id, closest_ids, closest_distances, dmso_distance in results:
        group = h5f.create_group(molecule_id)
        group.create_dataset('closest_ids', data=closest_ids.astype('S'))  # Save IDs as strings
        group.create_dataset('distances', data=closest_distances)
        group.create_dataset('dmso_distance', data=dmso_distance)

In [ ]:
import h5py

# Parameters
molecule_id = 'JCP2022_080538'  # Example molecule ID

# Access the HDF5 file
with h5py.File(output_file, 'r') as h5f:
    # Check if the molecule_id exists in the HDF5 file
    if molecule_id in h5f:
        print(f"Molecule ID {molecule_id} found.")
        
        # Access the datasets
        closest_ids = h5f[f'{molecule_id}/dmso_distance'][:].astype(str)  # Convert bytes to strings
        distances = h5f[f'{molecule_id}/distances'][:]
        
        # Print the results
        print(f'Closest molecules to {molecule_id}:')
        print('IDs:', closest_ids)
        print('Distances:', distances)
    else:
        print(f"Molecule ID {molecule_id} not found in the HDF5 file.")


In [ ]:
import h5py

# File path
H5_DISTANCE_FILE = output_file

# Query molecule ID
query_id = "JCP2022_080538"  # Replace with the molecule ID you want to query

# Open the HDF5 file and retrieve the `dmso_distance`
with h5py.File(H5_DISTANCE_FILE, 'r') as h5f:
    if query_id in h5f:
        dmso_distance = h5f[f"{query_id}/dmso_distance"][()]
        print(f"DMSO distance for {query_id}: {dmso_distance}")
    else:
        print(f"Molecule ID {query_id} not found in the HDF5 file.")


# Convert to pg10

# Add molecular ids (zinc, chembl, pubchem)

In [ ]:
data[data['Metadata_InChI']=='InChI=1S/C2H6OS/c1-4(2)3/h1-2H3']

In [ ]:
data_test = data.sample(n=100)

In [ ]:
data_test

In [ ]:
# Add new columns for metadata (initialize them with default values, e.g., None)
data['Zinc_id'] = None  # Replace None with the logic to populate Zinc ids if available
data['Canonical_SMILES'] = None  # Replace None with the logic to populate Canonical SMILES if available
data['PubChem_id'] = None  # Replace None with the logic to populate PubChem ids if available



In [ ]:

updated_parquet_file_path = "updated_file.parquet"  # Replace with your desired output file path
data.to_parquet(updated_parquet_file_path, index=False)

print("Metadata columns added successfully!")
